In [1]:
import torch
from modeling_layoutlm import LayoutLMForTokenClassification
from transformers import (
    BertConfig,
    BertTokenizer,
)
from utils_docvqa import (
    read_docvqa_examples,
    convert_examples_to_features)
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from transformers.data.processors.squad import SquadResult
from tqdm import tqdm
import numpy as np
from tabulate import tabulate

/Users/hari/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bounding_box import save_bounding_boxes
SAMPLE_DATA = save_bounding_boxes(
    image_path="fsgj0223_84.png",
    questions=[
        "What is the title of the document?",
        "What is the numerical interest?"
    ]
)

In [3]:
MODEL_FOLDER = "./models/"
LABELS = ["start","end"]
pad_token_label_id=-100
labels = ["start","end"]
max_seq_length = 512
max_query_length = 64
doc_stride = 128

In [4]:
# device = torch.device("cuda:0")
device = torch.device("cpu")
# torch.cuda.set_device(device)
model_class = LayoutLMForTokenClassification
config_class = BertConfig
tokenizer_class = BertTokenizer
config = config_class.from_pretrained(MODEL_FOLDER,num_labels=2,cache_dir=None)
model = model_class.from_pretrained(MODEL_FOLDER)
tokenizer = tokenizer_class.from_pretrained(MODEL_FOLDER,do_lower_case=True)

In [5]:
examples = read_docvqa_examples(SAMPLE_DATA, is_training=False)

In [6]:
features = convert_examples_to_features(
            examples=examples,
            label_list=labels,
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            pad_token_label_id=pad_token_label_id)

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 1000000000
INFO:tensorflow:example_index: 0
INFO:tensorflow:doc_span_index: 0
INFO:tensorflow:tokens: [CLS] what is the title of the document ? [SEP] @ tt . management report of the directors an discussion & analysis your directors here ##by submit their forty eighth annual report with the audit ##ed accounts for the year ended 31 " march , 2004 . summary of financial result : for the year for the year ended ended 31 . 03 . 2004 31 . 03 . 2003 ( rs . in lac ##s ) profit for the year before interest , de ##pre ##ciation and exceptional items : 296 ##1 . 59 269 ##7 . 42 interest 41 ##45 / 90 104 ##7 - 79 de ##pre ##ciation 240 . 76 281 . 73 profit bo ##for ##e tax and exceptional items 1874 . 93 136 ##7 . 90 exceptional te ##ms ( 289 . 65 ) ( 48 ##4 . 68 ) de ##pre ##ciation of eat ##lier yo ##ars — ( 145 . 94 ) prof ##il before taxation 128 ##5 . 28 76 ##7 . 28 provision for taxation current tax ( wealth tax ) 5 . 75 5 . 09 dete

In [7]:
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_bboxes = torch.tensor([f.boxes for f in features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)

eval_dataset = TensorDataset(
        all_input_ids, all_input_mask, all_segment_ids,all_bboxes,all_example_index)
eval_batch_size = 1
eval_sampler = (
        SequentialSampler(eval_dataset))

eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size
    )

In [8]:
model.to(device)
all_results = []
table_data = []

def to_list(tensor):
    return tensor.detach().cpu().tolist()

for batch in tqdm(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
        }
        inputs["bbox"] = batch[3]
        inputs["token_type_ids"] = (batch[2])
        outputs = model(**inputs)
    example_indices = batch[4]

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)

        output = [to_list(output[i]) for output in outputs]

        start_logits, end_logits = output
        result = SquadResult(unique_id, start_logits, end_logits)
        all_results.append(result)
predictions_json = {}
assert len(all_results)==len(features)
for i in range(len(all_results)):
    start_index = np.argmax(all_results[i].start_logits)
    end_index = np.argmax(all_results[i].end_logits)
    pred_answer = features[i].tokens[start_index:end_index+1]
    pred_answer = ' '.join([x for x in pred_answer])
    pred_text = pred_answer.replace(' ##', '')
    question = features[i].tokens[1:features[i].tokens.index('[SEP]')]
    question_text = ' '.join([x for x in question])
    question_text = question_text.replace(' ##', '')
    table_data.append([question_text, pred_text])
    # print(question_text)
    # print(pred_text) 


headers = ["Question", "Answer"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

Evaluating: 100%|██████████| 7/7 [00:01<00:00,  3.64it/s]

+-------------------------------------+-------------------------------------------------------------+
| Question                            | Answer                                                      |
+=====================================+=============================================================+
| what is the title of the document ? | management report of the directors an discussion & analysis |
+-------------------------------------+-------------------------------------------------------------+
| what is the title of the document ? | [CLS]                                                       |
+-------------------------------------+-------------------------------------------------------------+
| what is the title of the document ? | [CLS]                                                       |
+-------------------------------------+-------------------------------------------------------------+
| what is the title of the document ? | [CLS]                                     